In [25]:
import psycopg2
import pprint
from psycopg2.extras import RealDictCursor
import pandas as pd

DB_SETTINGS = {
    "host": "82.25.70.181",
    "port": 25432,
    "user": "postgres",
    "password": "Yp82FEfJGsMG4FIw",
    "dbname": "pediatriadb_fake",
}

def execute_query(query: str, params: tuple | None = None, fetch: bool = True):
    """Run the given SQL query against the configured Postgres database."""
    # Use context managers to guarantee the connection and cursor close correctly.
    with psycopg2.connect(**DB_SETTINGS) as conn:
        with conn.cursor(cursor_factory=RealDictCursor) as cursor:
            cursor.execute(query, params)
            # Return rows only when the caller requests data (e.g., SELECT statements).
            if fetch:
                return cursor.fetchall()

result = execute_query("SELECT NOW() AS server_time;")
pprint.pprint(result)

[RealDictRow([('server_time',
               datetime.datetime(2025, 10, 31, 17, 27, 13, 801727, tzinfo=datetime.timezone.utc))])]


In [ ]:
# Obtener top 5 pacintes con mas evoluciones
sql_query_paciente = f"""
select p.nombre, p.apellido, p.origin_id, p.id_paciente, count(*) as tot_consultas from "Pacientes" p
JOIN "Evoluciones" e ON p.origin_id = e.origin_id_paciente
GROUP BY p.nombre, p.apellido, p.origin_id, p.id_paciente
ORDER BY tot_consultas DESC LIMIT 5;
"""
top5_evoluciones_df = pd.execute_query(sql_query_paciente)
pprint.pprint(top5_evoluciones_df.to_markdown())

In [27]:
# Obtener datos personales del paciente
id_paciente = "497b04b7-1ccc-461f-b2a5-290f8af24f3b"
sql_query_paciente = f"""
SELECT t.*
FROM "Pacientes" t
WHERE t.id_paciente::text LIKE '{id_paciente}';
"""
paciente_df = execute_query(sql_query_paciente)

# Obtener evolucion de un paciente
origin_id_paciente = paciente_df[0]['origin_id']
sql_query_evolucion = f"""
SELECT t.*
FROM "Evoluciones" t
WHERE t.origin_id_paciente = '{origin_id_paciente}';
"""
evolucion_df = execute_query(sql_query_evolucion)


In [34]:
# configurar el parser de salida
from typing import Optional
from pydantic import BaseModel, Field
from uuid import UUID
from datetime import date, datetime

class Paciente(BaseModel):
    id_paciente: UUID = Field(...)  # Cambié de int a UUID
    origin_id: int = Field(...)
    apellido: str = Field(...)
    nombre: str = Field(...)
    fecha_nac: date = Field(...)  # Cambié de str a date
    sexo: str = Field(...)
    edad: str = Field(...)
    dni: Optional[str] = Field(None)
    localidad: Optional[str] = Field(None)
    obra_social: Optional[str] = Field(None)
    afiliado_nro: Optional[str] = Field(None)
    telefono: Optional[str] = Field(None)
    telefono_numero: Optional[int] = Field(None)  # Cambié de str a int
    email: Optional[str] = Field(None)
    especialidad: str = Field(...)
    diagnostico: Optional[str] = Field(None)
    enfermedad_base: Optional[str] = Field(None)
    ant_perinatales: Optional[str] = Field(None)
    ant_familiares: Optional[str] = Field(None)
    usuario_registro: str = Field(...)  # Cambié de 'registro' a 'usuario_registro'
    fecha_reg: Optional[date] = Field(None)  # Cambié de 'fecha_registro' a 'fecha_reg'
    createdAt: datetime = Field(...)
    updatedAt: datetime = Field(...)
    
    
class Evolucion(BaseModel):
    id_evolucion: UUID = Field(...)
    origin_id: int = Field(...)
    origin_id_paciente: int = Field(...)
    fecha_evolucion: date = Field(...)
    evolucion: str = Field(...)
    file: Optional[str] = Field(None)
    createdAt: datetime = Field(...)
    updatedAt: datetime = Field(...)
    
class Consulta(BaseModel):
    id_consulta: UUID = Field(...)
    origin_id: int = Field(...)
    origin_id_paciente: int = Field(...)
    fecha_consulta: date = Field(...)
    edad: int = Field(...)
    uni_edad: str = Field(...)
    edad_anios: float = Field(...)
    edad_texto: str = Field(...)
    peso: Optional[float] = Field(None)
    talla: Optional[float] = Field(None)
    imc: Optional[float] = Field(None)
    pc: Optional[float] = Field(None)
    motivo: str = Field(...)
    conducta: str = Field(...)
    createdAt: datetime = Field(...)
    updatedAt: datetime = Field(...)

In [35]:
# instanciando paciente y array de evoluciones con sus datos
paciente_df = pd.DataFrame(paciente_df)
evolucion_df = pd.DataFrame(evolucion_df)

paciente = Paciente(**paciente_df.to_dict(orient='records')[0])
evoluciones = [Evolucion(**item) for item in evolucion_df.to_dict(orient='records')]

# pprint.pprint(paciente)
# pprint.pprint(evoluciones)

In [36]:
#%pip install langchain-openai
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    input=["paciente", "evoluciones"],
    template="""
    ### preparacion de datos de paciente y evoluciones
        Paciente: {paciente}
        Evoluciones: {evoluciones}
    ### Preparacion de Prompt
        Eres un asistente que ayuda a redactar notas medicas a partir de los datos del paciente y sus evoluciones. Tu tarea es generar un resumen de la historia clinica del paciente, incluyendo los datos relevantes de las evoluciones.
        Debes tener en cuenta:
            - que las evoluciones pueden tener diferentes formatos y contenidos, por lo que debes ser capaz de interpretar y sintetizar la informacion de manera coherente.
            - cruzar informacion de las diferentes evoluciones para identificar patrones, cambios y tendencias en la salud del paciente.
            - cruzar los antecendentes personales y familiares del paciente, asi como su diagnostico y enfermedad base.
    ### Salida esperada:
        Se espera que la salida sea un texto en pocos parrafos.
        El texto debe ser claro y conciso, utilizando lenguaje medico apropiado.
        Al comenzar la redaccion, incluye una breve introduccion con los datos personales del paciente (nombre, edad, diagnostico -si es que lo tiene-, enfermedad base -si es que lo tiene-).
        Cada patron, tendencia o cambio identificado debe ser explicado de manera clara y breve precisa en un parrafo aparte, utilizando como dato de referencia la fecha de la evolucion en la que se toma la informacion.
        No debes:
            - enumerar las evoluciones ni copiar literalmente su contenido.
            - incluir informacion sobre obras sociales ni numero de documento. 
            - confundir el termino "Control de salud" con diagnosticos ni enfermedades. simplemente obvia ese dato si es que aparece en las evoluciones.
            - confundir "Otra" con diagnosticos ni enfermedades. simplemente obvia ese dato si es que aparece en las evoluciones.
            - inventar informacion que no este presente en los datos del paciente y sus evoluciones.
    """,
    output_parser=StrOutputParser()
)

In [37]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

# Configurar la API key de OpenAI
openai_api_key = os.getenv("OPENAI_API_KEY")
llm_model = "gpt-4o-mini"
llm = ChatOpenAI(openai_api_key=openai_api_key, model = llm_model, temperature=0.7)

In [38]:
chain = prompt | llm | StrOutputParser()
input_data = {
    "paciente": paciente,
    "evoluciones": evoluciones
}
respuesta = chain.invoke(input = input_data)
pprint.pprint(respuesta)

('Isabella Rossi, una paciente de 11 años, presenta un diagnóstico de Púrpura '
 'Trombocitopénica Inmunológica (PTI) y tiene como enfermedad base un estudio '
 'molecular confirmatorio de mutación en el gen MYH9, con variante de '
 'significado incierto. Isabella tiene antecedentes perinatales que incluyen '
 'un nacimiento a término sin internaciones, y antecedentes familiares '
 'relevantes que indican que su padre padece tiroiditis de Hashimoto.\n'
 '\n'
 'A lo largo de su seguimiento, se ha observado que el recuento plaquetario de '
 'Isabella ha fluctuado significativamente. En enero de 2024, su peso era de '
 '42.9 kg, y los recuentos plaquetarios estaban en torno a 50,000/mm³, lo que '
 'motivó el inicio de un tratamiento con eltrombopag. A partir de octubre de '
 '2024, se decidió un descenso en la dosis de eltrombopag debido a la '
 'estabilización de sus recuentos plaquetarios, aunque todavía se mantenía un '
 'control regular.\n'
 '\n'
 'En enero de 2025, se realizó un cent

In [40]:
# Opción 1: Mostrar como DataFrame (más legible)
import pandas as pd
df_visual = pd.DataFrame(paciente_df)
print("=== DATOS DEL PACIENTE ===")
display(df_visual)

# Opción 2: Mostrar campos específicos del paciente de forma estructurada
print("\n=== INFORMACIÓN DEL PACIENTE ===")
paciente_data = paciente_df[0]  # Primer registro
print(f"📋 Nombre: {paciente_data['nombre']} {paciente_data['apellido']}")
print(f"🎂 Fecha de Nacimiento: {paciente_data['fecha_nac']}")
print(f"⚥ Sexo: {paciente_data['sexo']}")
print(f"🆔 ID Paciente: {paciente_data['id_paciente']}")
print(f"📍 Localidad: {paciente_data.get('localidad', 'No especificada')}")
print(f"🏥 Obra Social: {paciente_data.get('obra_social', 'No especificada')}")
print(f"🩺 Especialidad: {paciente_data['especialidad']}")
print(f"📝 Diagnóstico: {paciente_data.get('diagnostico', 'No especificado')}")

# Opción 3: JSON pretty print
print("\n=== DATOS COMPLETOS (JSON) ===")
import json
print(json.dumps(paciente_data, indent=2, default=str, ensure_ascii=False))

=== DATOS DEL PACIENTE ===


,id_paciente,origin_id,apellido,nombre,fecha_nac,sexo,edad,dni,localidad,obra_social,...,email,especialidad,diagnostico,enfermedad_base,ant_perinatales,ant_familiares,usuario_registro,fecha_reg,createdAt,updatedAt
0,497b04b7-1ccc-461f-b2a5-290f8af24f3b,86,Rossi,Isabella,2013-03-25,,11 Años,53.065.334,Olavarría,IOMA,...,,Hematológico,PTI,estudio molecular confirmatorio de mutación en...,RNT PAEG No internaciones de nacimiento\r\nHAV...,Padre: Tiroiditis de Hashimoto,Administrador,2024-10-17,2024-10-17 00:00:00+00:00,2025-10-24 16:23:54.292832+00:00



=== INFORMACIÓN DEL PACIENTE ===


KeyError: 0